# Requirements :

In [1]:
%pip install PyMySQL

     ---------------------------------------- 43.8/43.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Import the required libraries

In [1]:
import pymysql

# Connexion to the database instance

In [2]:
db = pymysql.connect(
        host='IP-ADDRESS',
        user='USER', 
        password = "PASSWORD",
        )

In [3]:
cursor = db.cursor()

In [4]:
cursor.execute("select version()")
data = cursor.fetchone()
print("Database version : %s " % data)

Database version : 8.0.28 


# Create Database

In [13]:
cursor.execute("CREATE DATABASE IF NOT EXISTS `ESGI`")

1

In [15]:
cursor.connection.commit()

In [6]:
cursor.execute("USE ESGI")

0

# Create Table

In [34]:
sql = """
CREATE TABLE IF NOT EXISTS bookreview (
    id int(11) NOT NULL AUTO_INCREMENT,
    user_id varchar(50) NULL,
    book_id int(25) NULL,
    review_id varchar(50) NULL,
    rating tinyint(1) NULL,
    review_text varchar(2000) NULL,
    date_added varchar(40) NULL,
    date_updated varchar(40) NULL,
    read_at varchar(40) NULL,
    started_at varchar(40) NULL,
    n_votes smallint(4) NULL,
    n_comments smallint(4) NULL,
    PRIMARY KEY (id)
)"""

cursor.execute(sql)

0

In [7]:
cursor.execute("SHOW TABLES")
cursor.fetchall()

(('bookreview',),)

In [8]:
cursor.execute("desc ESGI.bookreview")
cursor.fetchall()

(('id', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('user_id', 'varchar(50)', 'YES', '', None, ''),
 ('book_id', 'int', 'YES', '', None, ''),
 ('review_id', 'varchar(50)', 'YES', '', None, ''),
 ('rating', 'tinyint(1)', 'YES', '', None, ''),
 ('review_text', 'varchar(2000)', 'YES', '', None, ''),
 ('date_added', 'varchar(40)', 'YES', '', None, ''),
 ('date_updated', 'varchar(40)', 'YES', '', None, ''),
 ('read_at', 'varchar(40)', 'YES', '', None, ''),
 ('started_at', 'varchar(40)', 'YES', '', None, ''),
 ('n_votes', 'smallint', 'YES', '', None, ''),
 ('n_comments', 'smallint', 'YES', '', None, ''))

# Feed the Bookreview Table

In [9]:
import pandas as pd
import tqdm

### Create a dataset based on goodreads_train.csv

In [10]:
dataset = pd.read_csv('../data/goodreads_train.csv', sep=',')
dataset.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


### Refractor all the 'NaN' value to None (Understand by MySQL)

In [11]:
df1 = dataset.where(pd.notnull(dataset), None)
df1.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,None,None,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,None,None,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


### Loop on the dataset and add the value to the ESGI.bookreview table

In [24]:
for i in tqdm.tqdm(range(100)): #len(df1)
    sql = "INSERT INTO bookreview (user_id, book_id, review_id, rating, review_text, date_added, date_updated, read_at, started_at, n_votes, n_comments) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, tuple(df1.iloc[i]))
    db.commit()

100%|██████████| 100/100 [00:02<00:00, 37.31it/s]


### Preview of the ESGI.bookreview content

In [25]:
cursor.execute("SELECT * FROM bookreview")
cursor.fetchall()

((1,
  '8842281e1d1347389f2ab93d60773d4d',
  18245960,
  'dfdbb7b0eb5a7e4c26d59a937e2e5feb',
  5,
  'This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. (view spoiler)[Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though. \n But what would happen if our SETI stations received 

In [27]:
select = cursor.execute("SELECT * FROM bookreview")
select = cursor.fetchall()
select = pd.DataFrame(select)
select.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,2,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,None,None,1,0
2,3,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,4,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,None,None,5,1
4,5,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1
